## 7. Ingest pit_stops.json file

In [0]:
%fs
ls /mnt/prisha12/raw/

### 1. read the JSON file using the spark dataframe reader API

In [0]:
from pyspark.sql.types import IntegerType, StringType, FloatType, TimestampType, StructType, StructField

pit_stops_schema = StructType([StructField('raceId', IntegerType(), False),
                               StructField('driverId', IntegerType(), True),
                               StructField('stop', IntegerType(), True),
                               StructField('lap', IntegerType(), True),
                               StructField('time', StringType(), True),
                               StructField('duration', StringType(),True),
                               StructField('milliseconds', IntegerType(), True)])

In [0]:
pit_stops_df = spark.read.schema(pit_stops_schema).option("multiLine",True).json("/mnt/prisha12/raw/pit_stops.json")

### 2. Rename columns and add new columns: ingestion time

In [0]:
## ingesting time, need current timestamp
from pyspark.sql.functions import current_timestamp
pit_stops_selected = pit_stops_df.withColumnRenamed('driverId','driver_id').withColumnRenamed('raceId','race_id').withColumn('ingesting_timestamp', current_timestamp())

### 3. Write to output to processed container in parquet format

In [0]:
pit_stops_selected.write.mode("overwrite").format('parquet').save('/mnt/prisha12/processed/pit_stops')

In [0]:
display(spark.read.parquet('/mnt/prisha12/processed/pit_stops'))